In [111]:
import pandas as pd
import os
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
from scipy.stats import mode
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
#import xgboost as xgb
#from xgboost import XGBClassifier
#from xgboost import plot_importance
#from matplotlib import pyplot

In [112]:
def _log(str):
    os.system(f'echo \"{str}\"')
    print(str)

In [113]:
INPUT_ROOT = '../input/data-science-bowl-2019'
JOIN_KEY = ['installation_id', 'game_session', 'title']
TARGET = 'accuracy_group'
FEATURES = {
    'event_id', 
    'game_session', 
    'timestamp', 
    'installation_id', 
    'event_count',
    'event_code', 
    'game_time', 
    'title', 
    'type', 
    'world'
}
CATEGORICAL_FEATURES = {
    'event_id', 
    'game_session',
    'timestamp',
    'installation_id',
    'event_code',
    'title', 
    'type', 
    'world'
}

In [114]:
def _init():
    # Characters such as empty strings '' or numpy.inf are considered NA values
    pd.set_option('use_inf_as_na', True)
    pd.set_option('display.max_columns', 500)
    
    
_init()

In [115]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk(INPUT_ROOT):
    for filename in filenames:
        _log(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

../input/data-science-bowl-2019\test.csv
../input/data-science-bowl-2019\test.csv.zip
../input/data-science-bowl-2019\train.csv
../input/data-science-bowl-2019\train.csv.zip
../input/data-science-bowl-2019\train_labels.csv
../input/data-science-bowl-2019\train_labels.csv.zip


In [116]:
%%time
train_raw = pd.read_csv(f'{INPUT_ROOT}/train.csv', usecols=FEATURES)
train_labels = pd.read_csv(f'{INPUT_ROOT}/train_labels.csv', usecols=JOIN_KEY + [TARGET])
test = pd.read_csv(f'{INPUT_ROOT}/test.csv', usecols=FEATURES)
train_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11341042 entries, 0 to 11341041
Data columns (total 10 columns):
event_id           object
game_session       object
timestamp          object
installation_id    object
event_count        int64
event_code         int64
game_time          int64
title              object
type               object
world              object
dtypes: int64(3), object(7)
memory usage: 865.3+ MB
Wall time: 37.9 s


In [117]:
train_labels.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17690 entries, 0 to 17689
Data columns (total 4 columns):
game_session       17690 non-null object
installation_id    17690 non-null object
title              17690 non-null object
accuracy_group     17690 non-null int64
dtypes: int64(1), object(3)
memory usage: 552.9+ KB


In [118]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1156414 entries, 0 to 1156413
Data columns (total 10 columns):
event_id           1156414 non-null object
game_session       1156414 non-null object
timestamp          1156414 non-null object
installation_id    1156414 non-null object
event_count        1156414 non-null int64
event_code         1156414 non-null int64
game_time          1156414 non-null int64
title              1156414 non-null object
type               1156414 non-null object
world              1156414 non-null object
dtypes: int64(3), object(7)
memory usage: 88.2+ MB


In [119]:
vs = sorted(train_raw['title'].unique())
_log(f'{len(vs)} train_raw titles={vs}')

44 train_raw titles=['12 Monkeys', 'Air Show', 'All Star Sorting', 'Balancing Act', 'Bird Measurer (Assessment)', 'Bottle Filler (Activity)', 'Bubble Bath', 'Bug Measurer (Activity)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Chicken Balancer (Activity)', 'Chow Time', 'Costume Box', 'Crystal Caves - Level 1', 'Crystal Caves - Level 2', 'Crystal Caves - Level 3', 'Crystals Rule', 'Dino Dive', 'Dino Drink', 'Egg Dropper (Activity)', 'Fireworks (Activity)', 'Flower Waterer (Activity)', 'Happy Camel', 'Heavy, Heavier, Heaviest', 'Honey Cake', 'Leaf Leader', 'Lifting Heavy Things', 'Magma Peak - Level 1', 'Magma Peak - Level 2', 'Mushroom Sorter (Assessment)', 'Ordering Spheres', 'Pan Balance', "Pirate's Tale", 'Rulers', 'Sandcastle Builder (Activity)', 'Scrub-A-Dub', 'Slop Problem', 'Treasure Map', 'Tree Top City - Level 1', 'Tree Top City - Level 2', 'Tree Top City - Level 3', 'Watering Hole (Activity)', 'Welcome to Lost Lagoon!']


In [120]:
vs = sorted(test['title'].unique())
_log(f'{len(vs)} test titles={vs}')

44 test titles=['12 Monkeys', 'Air Show', 'All Star Sorting', 'Balancing Act', 'Bird Measurer (Assessment)', 'Bottle Filler (Activity)', 'Bubble Bath', 'Bug Measurer (Activity)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Chicken Balancer (Activity)', 'Chow Time', 'Costume Box', 'Crystal Caves - Level 1', 'Crystal Caves - Level 2', 'Crystal Caves - Level 3', 'Crystals Rule', 'Dino Dive', 'Dino Drink', 'Egg Dropper (Activity)', 'Fireworks (Activity)', 'Flower Waterer (Activity)', 'Happy Camel', 'Heavy, Heavier, Heaviest', 'Honey Cake', 'Leaf Leader', 'Lifting Heavy Things', 'Magma Peak - Level 1', 'Magma Peak - Level 2', 'Mushroom Sorter (Assessment)', 'Ordering Spheres', 'Pan Balance', "Pirate's Tale", 'Rulers', 'Sandcastle Builder (Activity)', 'Scrub-A-Dub', 'Slop Problem', 'Treasure Map', 'Tree Top City - Level 1', 'Tree Top City - Level 2', 'Tree Top City - Level 3', 'Watering Hole (Activity)', 'Welcome to Lost Lagoon!']


In [121]:
vs = sorted(train_labels['title'].unique())
_log(f'{len(vs)} train_labels titles={vs}')

5 train_labels titles=['Bird Measurer (Assessment)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Mushroom Sorter (Assessment)']


In [122]:
def _remove_unlabelled_data(train_raw, train_labels):
    return train_raw[train_raw['installation_id'].isin(train_labels['installation_id'].unique())]


train_raw = _remove_unlabelled_data(train_raw, train_labels)

In [123]:
%%time
def _add_labels(train_raw, train_labels, on):
    return pd.merge(train_raw, train_labels, on=on, how='left')


train = _add_labels(train_raw, train_labels, on=JOIN_KEY)
del train_raw
#_log(f'train[\'{TARGET}\'] count null={train[TARGET].isna().sum()}')

Wall time: 3.74 s


In [124]:
def _transform_timestamp(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df


train = _transform_timestamp(train)
test = _transform_timestamp(test)

In [125]:
def _drop_unused_columns(df):
    cols = ['game_session', 'type']
    return df.drop(columns=cols)

    
#train = _drop_unused_columns(train)
#test = _drop_unused_columns(test)

In [126]:
%%time
def _set_string_type(df, cols):
    df[cols] = df[cols].astype(str)
    return df


cols = list(CATEGORICAL_FEATURES)
train = _set_string_type(train, cols=cols + [TARGET])
test = _set_string_type(test, cols=cols)

Wall time: 1min 38s


In [127]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7734558 entries, 0 to 7734557
Data columns (total 11 columns):
event_id           object
game_session       object
timestamp          object
installation_id    object
event_count        int64
event_code         object
game_time          int64
title              object
type               object
world              object
accuracy_group     object
dtypes: int64(2), object(9)
memory usage: 708.1+ MB


In [128]:
def _log_smoothing(df, cols):
    for col in cols:
        df[col] = np.log(df[col] + 1)
    return df


#cols = ['event_count', 'game_time']
#train = _log_smoothing(train, cols)
#test = _log_smoothing(test, cols)

In [129]:
train.head()

,event_id,game_session,timestamp,installation_id,event_count,event_code,game_time,title,type,world,accuracy_group
0,27253bdc,34ba1a28d02ba8ba,2019-08-06 04:57:18.904000+00:00,0006a69f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,nan
1,27253bdc,4b57c9a59474a1b9,2019-08-06 04:57:45.301000+00:00,0006a69f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,nan
2,77261ab5,2b9d5af79bcdb79f,2019-08-06 04:58:14.538000+00:00,0006a69f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,nan
3,b2dba42b,2b9d5af79bcdb79f,2019-08-06 04:58:14.615000+00:00,0006a69f,2,3010,29,Sandcastle Builder (Activity),Activity,MAGMAPEAK,nan
4,1325467d,2b9d5af79bcdb79f,2019-08-06 04:58:16.680000+00:00,0006a69f,3,4070,2137,Sandcastle Builder (Activity),Activity,MAGMAPEAK,nan


In [130]:
train.describe(include='all')

,event_id,game_session,timestamp,installation_id,event_count,event_code,game_time,title,type,world,accuracy_group
count,7734558,7734558,7734558,7734558,7.734558e+06,7734558,7.734558e+06,7734558,7734558,7734558,7734558
unique,379,175467,7347901,3614,NaN,42,NaN,44,4,4,5
top,bb3e370b,bb1f09ec062b6660,2019-09-20 19:45:56.536000+00:00,f1c21eda,NaN,4070,NaN,Bottle Filler (Activity),Game,MAGMAPEAK,nan
freq,168955,3182,10,58988,NaN,1521039,NaN,661294,3834750,3208242,6869111
mean,NaN,NaN,NaN,NaN,9.775152e+01,NaN,1.677046e+05,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,1.311293e+02,NaN,1.068495e+06,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,1.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,2.400000e+01,NaN,2.927100e+04,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,5.700000e+01,NaN,6.941200e+04,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,1.210000e+02,NaN,1.479110e+05,NaN,NaN,NaN,NaN


In [131]:
test.head()

,event_id,game_session,timestamp,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10 16:50:24.910000+00:00,00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10 16:50:55.503000+00:00,00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10 16:51:51.805000+00:00,00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10 16:53:12.825000+00:00,00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10 16:54:12.115000+00:00,00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [132]:
test.describe(include='all')

,event_id,game_session,timestamp,installation_id,event_count,event_code,game_time,title,type,world
count,1156414,1156414,1156414,1156414,1.156414e+06,1156414,1.156414e+06,1156414,1156414,1156414
unique,365,28445,1103166,1000,NaN,42,NaN,44,4,4
top,bb3e370b,b246933e8ac6a4aa,2019-09-13 18:38:38.848000+00:00,7b728c89,NaN,4070,NaN,Bottle Filler (Activity),Game,MAGMAPEAK
freq,28667,4456,9,21237,NaN,234260,NaN,112223,572260,511291
mean,NaN,NaN,NaN,NaN,1.107075e+02,NaN,1.869646e+05,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,2.193408e+02,NaN,1.670162e+06,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,1.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,2.500000e+01,NaN,3.056125e+04,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,6.000000e+01,NaN,7.102800e+04,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,1.240000e+02,NaN,1.476910e+05,NaN,NaN,NaN


In [133]:
#train.groupby(['installation_id', 'title'])['accuracy_group'].nunique()

# Train-test split not by time
Both train and test sets span the same time period.

In [135]:
_log(f'train[timestamp] is from {train.timestamp.min()} to {train.timestamp.max()}')
_log(f'test[timestamp] is from {test.timestamp.min()} to {test.timestamp.max()}')

train[timestamp] is from 2019-07-23 14:38:25.256000+00:00 to 2019-10-14 21:57:26.930000+00:00
test[timestamp] is from 2019-07-24 00:04:25.361000+00:00 to 2019-10-14 21:00:34.858000+00:00
